In [ ]:
import csv
import PIL
from PIL import Image
from tensorflow import keras
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw, ImageTk 
from tkinter import *
from tkinter import filedialog

csvfiles = {
    1: 'UFPR04.csv',
    2: 'UFPR05.csv',
    3: 'PUCPR.csv'
}
root = Tk()
root.title('Select an image')
root.geometry("1070x740")

def browseFile():
    global myIimage
    
    root.filename = filedialog.askopenfilename(initialdir='/', title='Select a file',
                                               filetypes=(('JPG files', '*.jpg'),
                                                          ('All files', '*.*')))
    img = PIL.Image.open(root.filename)
    
    
    UFPR04parking(img)

def UFPR04parking(im):
    global model
    print(f"{csvfiles[v.get()]} selected")
        
    spots_csv = open(csvfiles[v.get()], mode='r')
    csv_reader = csv.DictReader(spots_csv)

    occupied = []
    available = []
    model = tf.keras.models.load_model('XXX.h5')
    draw = ImageDraw.Draw(im)
    for row in csv_reader:
        spot_img = im.crop((int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])))
        newsize = (40, 32)
        spot_img = spot_img.resize(newsize)
        # spot_img.show()                             # Comment Uncomment
        result = makePrediction(spot_img,model)
        if result == 1:
            occupied.append(row["id"])
            draw.rectangle([int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])], fill=None, outline='red', width=1)
        else:
            available.append(row["id"])
            draw.rectangle([int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])], fill=None, outline='green', width=1)
    print(f"{len(occupied)} occupied")
    print(f"{len(available)} available")
    print('These spots are available ', available)
    mytext = f"{len(occupied)} occupied, {len(available)} available \n These spots are available {available}"
    
    im = im.resize((1000, 563), PIL.Image.ANTIALIAS) 
    myImage = ImageTk.PhotoImage(im)
    myImagelabel = Label(root, text=mytext, font=("Helvetica", 16), image=myImage, compound='bottom')
    myImagelabel.image = myImage
    myImagelabel.pack()
        
    
def makePrediction(spot_img, model):
    img_array = keras.preprocessing.image.img_to_array(spot_img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    result = model.predict(img_array, steps=1)
    result = np.rint(result)
    if (result):
        return 1
    else:
        return 0
    
myButton = Button(root, text='Open File', command=browseFile).pack(pady=10)
v = IntVar()
v.set("L") # initialize
Radiobutton(root, text="Parking 1", variable=v, value=1).pack(anchor=CENTER)
Radiobutton(root, text="Parking 2", variable=v, value=2).pack(anchor=CENTER)
Radiobutton(root, text="Parking 3", variable=v, value=3).pack(anchor=CENTER)
root.mainloop()

UFPR05.csv selected
39 occupied
1 available
These spots are available  ['28']
PUCPR.csv selected
